In [2]:
pip install pip install torch torchvision torchaudio

ERROR: Could not find a version that satisfies the requirement install (from versions: none)
ERROR: No matching distribution found for install
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install deep-sort-realtime


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install ultralytics


Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install flask opencv-python

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
from flask import Flask, Response, render_template
import threading
from camera_functions import yolo_results, yolo_ds_draw, yolo_ds_model_initalize, yolo_ds_update
import socket

In [7]:
# Initialize Flask app
app = Flask(__name__)


In [8]:
model, class_names, tracker = yolo_ds_model_initalize(model_name = "yolo11n.pt", max_age= 30)

In [9]:
frame_lock = threading.Lock()
latest_frame = None
annotated_frame = None
last_detections = []
frame_count = 0

In [ ]:
def capture_and_process():
    global latest_frame, annotated_frame, last_detections, frame_count
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            continue

        frame_count += 1

        with frame_lock:
            latest_frame = frame.copy()

            if frame_count % 2 == 0:
                # Run YOLO detection every 2nd frame
                results = yolo_results(model, frame)
                last_detections = yolo_ds_update(frame, results, tracker)
            annotated_frame = yolo_ds_draw(frame, last_detections, class_names)

In [11]:
def get_ip_address():
    hostname = socket.gethostname()  # Get the hostname of the server
    ip_address = socket.gethostbyname(hostname)  # Resolve the IP address of the hostname
    return ip_address

In [ ]:
@app.route('/')
def index():
    ip_address = get_ip_address()
    return render_template('index_test_yolo.html', ip_address = ip_address)  # Serve the HTML page

# Function to generate video frames for Flask streaming
def generate():
    global annotated_frame
    while True:
        with frame_lock:
            if annotated_frame is None:
                continue
            ret, buffer = cv2.imencode('.jpg', annotated_frame)
            if not ret:
                continue
            frame_bytes = buffer.tobytes()

        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame_bytes + b'\r\n\r\n')

# Flask route to serve video stream
@app.route('/video_feed')
def video_feed():
    return Response(generate(), mimetype='multipart/x-mixed-replace; boundary=frame')


In [ ]:
if __name__ == '__main__':
    threading.Thread(target=capture_and_process, daemon=True).start()

    # Run Flask server
    app.run(host='0.0.0.0', port=4000, threaded=True)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:4000
 * Running on http://192.168.8.60:4000
Press CTRL+C to quit
2025-04-03 04:32:49.901 python[70079:14149909] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.



0: 384x640 (no detections), 113.2ms
Speed: 4.9ms preprocess, 113.2ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 tie, 72.8ms
Speed: 2.2ms preprocess, 72.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 tie, 85.5ms
Speed: 3.0ms preprocess, 85.5ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 tie, 49.8ms
Speed: 2.5ms preprocess, 49.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 tie, 86.5ms
Speed: 12.6ms preprocess, 86.5ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 tie, 66.4ms
Speed: 2.9ms preprocess, 66.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 tie, 50.2ms
Speed: 1.7ms preprocess, 50.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 tie, 96.4ms
Speed: 3.0ms preprocess, 96.4m

127.0.0.1 - - [03/Apr/2025 04:32:56] "GET / HTTP/1.1" 200 -
192.168.8.60 - - [03/Apr/2025 04:32:56] "GET /video_feed HTTP/1.1" 200 -



0: 384x640 1 person, 1 tie, 43.7ms
Speed: 1.7ms preprocess, 43.7ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 tie, 43.1ms
Speed: 2.0ms preprocess, 43.1ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 tie, 43.9ms
Speed: 1.4ms preprocess, 43.9ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 tie, 45.0ms
Speed: 1.5ms preprocess, 45.0ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 tie, 78.8ms
Speed: 1.7ms preprocess, 78.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 tie, 44.1ms
Speed: 2.2ms preprocess, 44.1ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 tie, 35.6ms
Speed: 1.7ms preprocess, 35.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 tie, 77.0ms
Speed: 7.6ms preprocess, 77.0ms i


0: 384x640 1 person, 1 tie, 1 chair, 87.1ms
Speed: 2.9ms preprocess, 87.1ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 tie, 1 chair, 54.4ms
Speed: 2.7ms preprocess, 54.4ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
